### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [18]:

import configparser, os
from langchain.chains import (RouterChain, SequentialChain, LLMChain, MultiRouteChain, ConversationChain)
from langchain.llms import OpenAI, OpenAIChat
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

conf = configparser.ConfigParser()
conf.read('../config.ini')
api_key = conf.get("Openai", "api_key")  # 在config.ini中配置自己的APIkey
os.environ["HTTP_PROXY"] = conf.get("Proxy", "HTTP_PROXY")  # 配置自己的代理
os.environ["HTTPS_PROXY"] = conf.get("Proxy", "HTTPS_PROXY")
chat_model = "gpt-3.5-turbo"
text_model = "text-davinci-003"

In [19]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""

math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biology_template = """你是一位很优秀的生物学家。
你很擅长用简单易懂方式回答关于生物学的知识，先简单叙述问题的概念，在举一个生活的例子说明。
这个问题是：{input}
"""

chinese_language_template = """你是一位擅长汉语语言文学的教授，擅长做答汉语语言方面的问题。
答题时候直接切中要害，直奔给分点，多给作品例子来回答，会增强说服力。
这个问题是：{input}
"""

computer_template = """你是一位计算机软件和硬件方面的专家，软件方面：擅长各种编程语言以及编程技巧，硬件方面：擅长各种硬件设施以及配置调优等。
解答下面的问题，解答时候，软件方面：1.写出代码 2.加上简单的注释，硬件方面简单的步骤说明。
这个问题是：{input}"""



In [20]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物学问题",
        "prompt_template": biology_template,
    }, {
        "name": "计算机",
        "description": "适用于回答计算机软件和硬件方面的问题",
        "prompt_template": computer_template,
    }, {
        "name": "语文",
        "description": "适用于回答汉语语言文学方面的问题",
        "prompt_template": chinese_language_template,
    },
]

In [21]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}
llm = OpenAI(openai_api_key=api_key)
for prompt in prompt_infos:
    name = prompt['name']
    description = prompt['description']
    prompt_template = prompt['prompt_template']
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
# print(destination_chains.keys())

default_chain = ConversationChain(llm=llm, output_key="text")

In [22]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

In [ ]:
# 这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [25]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
print(router_template)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物学问题
计算机: 适用于回答计算机软件和硬件方面的问题
语文: 适用于回答汉语语言文学方面的问题

<< INPUT >>


In [28]:
from langchain.chains import MultiPromptChain

chain = MultiPromptChain(
    router_chain=router_chain,
    default_chain= default_chain,
    destination_chains=destination_chains,
    verbose=True
)


In [29]:
chain.run("“床前明月光，意思地上霜”是谁写的？")



> Entering new MultiPromptChain chain...


/Users/yangshengqin/my_pyenv/AINote/lib/python3.10/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


语文: {'input': '谁写了“床前明月光，疑是地上霜”？'}
> Finished chain.


'\n这是李白的诗句，出自《静夜思》。《静夜思》是李白的代表作之一，诗中表达了他对家乡的思念，对爱的痴迷，也抒发了他对静夜的情感。这首诗也把秦淮河畔的景色描绘得淋漓尽致，如：“沉醉不知归路，入眠不知东西。”这首诗被称为“李白的绝句宝库”，其中'

In [30]:
chain.run("用python写一个归并排序")



> Entering new MultiPromptChain chain...
计算机: {'input': '用Python编写归并排序算法'}
> Finished chain.


'\n\n步骤一：定义一个归并排序函数，函数接受一个列表作为参数。\n\n步骤二：如果列表的长度小于或等于1，那么直接返回这个列表，因为它已经排序完成。\n\n步骤三：取列表中间的元素，将列表一分为二。\n\n步骤四：调用归并排序函数，对这两个分开的列表分别进行排序。\n\n步'